In [125]:
# 导入工具函数：根据状态和行为生成操作相关字典的键，显示字典的内容
from utils import str_key,display_dict
# 设置转移概率、奖励值以及读取方式
from utils import set_prob,set_reward,get_prob,get_reward
# 设置状态价值、策略概率以及读取它们的方法
from utils import set_value,set_pi,get_value,get_pi

In [126]:
# 构建马尔科夫环境
S = ['浏览手机中', '第一节课', '第二节课', '第三节课', '休息中']
A = ['浏览手机', '学习', '离开浏览', '泡吧', '退出学习']
R = {} # 奖励字典
P = {} # 状态概率转移字典
gamma = 1.0 # 衰减因子
# 根据示例数据设置状态转移概率和奖励，默认概率为1
set_prob(P, S[0], A[0], S[0]) # 浏 览 手 机 中 - 浏 览 手 机 -> 浏 览 手 机 中
set_prob(P, S[0], A[2], S[1]) # 浏 览 手 机 中 - 离 开 浏 览 -> 第 一 节 课
set_prob(P, S[1], A[0], S[0]) # 第 一 节 课 - 浏 览 手 机 -> 浏 览 手 机 中
set_prob(P, S[1], A[1], S[2]) # 第 一 节 课 - 学 习 -> 第 二 节 课
set_prob(P, S[2], A[1], S[3]) # 第 二 节 课 - 学 习 -> 第 三 节 课
set_prob(P, S[2], A[4], S[4]) # 第 二 节 课 - 退 出 学 习 -> 退 出 休 息
set_prob(P, S[3], A[1], S[4]) # 第 三 节 课 - 学 习 -> 退 出 休 息
set_prob(P, S[3], A[3], S[1], p = 0.2) # 第 三 节 课 - 泡 吧 -> 第 一 节 课
set_prob(P, S[3], A[3], S[2], p = 0.4) # 第 三 节 课 - 泡 吧 -> 第 二 节 课
set_prob(P, S[3], A[3], S[3], p = 0.4) # 第 三 节 课 - 泡 吧 -> 第 三 节 课

set_reward(R, S[0], A[0], -1) # 浏 览 手 机 中 - 浏 览 手 机 -> -1
set_reward(R, S[0], A[2], 0) # 浏 览 手 机 中 - 离 开 浏 览 -> 0
set_reward(R, S[1], A[0], -1) # 第 一 节 课 - 浏 览 手 机 -> -1
set_reward(R, S[1], A[1], -2) # 第 一 节 课 - 学 习 -> -2
set_reward(R, S[2], A[1], -2) # 第 二 节 课 - 学 习 -> -2
set_reward(R, S[2], A[4], 0) # 第 二 节 课 - 退 出 学 习 -> 0
set_reward(R, S[3], A[1], 10) # 第 三 节 课 - 学 习 -> 10
set_reward(R, S[3], A[3], 1) # 第 三 节 课 - 泡 吧 -> +1

MDP = (S, A, R, P, gamma)
print(MDP)

(['浏览手机中', '第一节课', '第二节课', '第三节课', '休息中'], ['浏览手机', '学习', '离开浏览', '泡吧', '退出学习'], {'浏览手机中_浏览手机': -1, '浏览手机中_离开浏览': 0, '第一节课_浏览手机': -1, '第一节课_学习': -2, '第二节课_学习': -2, '第二节课_退出学习': 0, '第三节课_学习': 10, '第三节课_泡吧': 1}, {'浏览手机中_浏览手机_浏览手机中': 1.0, '浏览手机中_离开浏览_第一节课': 1.0, '第一节课_浏览手机_浏览手机中': 1.0, '第一节课_学习_第二节课': 1.0, '第二节课_学习_第三节课': 1.0, '第二节课_退出学习_休息中': 1.0, '第三节课_学习_休息中': 1.0, '第三节课_泡吧_第一节课': 0.2, '第三节课_泡吧_第二节课': 0.4, '第三节课_泡吧_第三节课': 0.4}, 1.0)


In [127]:
print("----状 态 转 移 概 率 字 典 （矩 阵） 信 息:----")
display_dict(P)
print("----奖 励 字 典 （函 数） 信 息:----")
display_dict(R)

----状 态 转 移 概 率 字 典 （矩 阵） 信 息:----
浏览手机中_浏览手机_浏览手机中: 1.00

浏览手机中_离开浏览_第一节课: 1.00

第一节课_浏览手机_浏览手机中: 1.00

第一节课_学习_第二节课: 1.00

第二节课_学习_第三节课: 1.00

第二节课_退出学习_休息中: 1.00

第三节课_学习_休息中: 1.00

第三节课_泡吧_第一节课: 0.20

第三节课_泡吧_第二节课: 0.40

第三节课_泡吧_第三节课: 0.40

----奖 励 字 典 （函 数） 信 息:----
浏览手机中_浏览手机: -1.00

浏览手机中_离开浏览: 0.00

第一节课_浏览手机: -1.00

第一节课_学习: -2.00

第二节课_学习: -2.00

第二节课_退出学习: 0.00

第三节课_学习: 10.00

第三节课_泡吧: 1.00



In [128]:
# 使用随机策略，初始条件下所有状态的价值设为0，与状态转移概率和奖励函数一样，策略与价值也通过一个字典来维护
# S = ['浏览手机中', '第一节课', '第二节课', '第三节课', '休息中']
# A = ['浏览手机', '学习', '离开浏览', '泡吧', '退出学习']
# 设 置 行 为 策 略： pi(a|.) = 0.5
Pi = {}
set_pi(Pi, S[0], A[0], 0.5)  # 浏 览 手 机 中 - 浏 览 手 机
set_pi(Pi, S[0], A[2], 0.5) # 浏 览 手 机 中 - 离 开 浏 览
set_pi(Pi, S[1], A[0], 0.5) # 第 一 节 课 - 浏 览 手 机
set_pi(Pi, S[1], A[1], 0.5) # 第 一 节 课 - 学 习
set_pi(Pi, S[2], A[1], 0.5) # 第 二 节 课 - 学 习
set_pi(Pi, S[2], A[4], 0.5) # 第 二 节 课 - 退 出 学 习
set_pi(Pi, S[3], A[1], 0.5) # 第 三 节 课 - 学 习
set_pi(Pi, S[3], A[3], 0.5) # 第 三 节 课 - 泡 吧

In [129]:
print("----策 略 字 典 （矩 阵） 信 息:----")
display_dict(Pi)
# 初 始 时 价 值 为 空， 访 问 时 会 返 回0
print("----状 态 价 值 字 典 （矩 阵） 信 息:----")
V = {}
display_dict(V)

----策 略 字 典 （矩 阵） 信 息:----
浏览手机中_浏览手机: 0.50

浏览手机中_离开浏览: 0.50

第一节课_浏览手机: 0.50

第一节课_学习: 0.50

第二节课_学习: 0.50

第二节课_退出学习: 0.50

第三节课_学习: 0.50

第三节课_泡吧: 0.50

----状 态 价 值 字 典 （矩 阵） 信 息:----


In [130]:
def compute_q(MDP, V, s, a):
    """
    根 据 给 定 的MDP， 价 值 函 数V， 计算某一状态s时的某行为a的价值q(s,a)
    :param MDP:
    :param V:
    :param s:
    :param a:
    :return:
    """
    S, A, R, P ,gamma = MDP
    q_sa = get_reward(R,s,a)
    for s_prime in S:
        q_sa += gamma * get_prob(P, s, a, s_prime) * get_value(V, s_prime)
    return q_sa

def compute_v(MDP, V, Pi, s):
    """
    给 定MDP下 依 据 某 一 策 略Pi和 当 前 状 态 价 值 函 数V计 算 某 状 态s的 价 值
    :param MDP:
    :param V:
    :param Pi:
    :param s:
    :return:
    """
    S, A, R, P, gamma = MDP
    v_s = 0
    for a in A:
        v_s += get_pi(Pi, s, a) * compute_q(MDP, V, s, a)
    return v_s

In [131]:
def update_V(MDP, V, Pi):
    """
    根 据 当 前  策 略 使 用 回 溯 法 来 更 新 状态 价 值
    :param MDP:
    :param V:
    :param Pi:
    :return:
    """
    S, _, _, _, _ = MDP
    V_prime = V.copy()
    for s in S:
        V_prime[str_key(s)] = compute_v(MDP, V_prime, Pi, s)
    return V_prime

def policy_evaluate(MDP, V, Pi, n):
    """
    策 略 评 估， 得 到 该 策 略 下 最 终 的 状 态 价 值。
    使 用n次 迭 代 计 算 来 评 估 一 个MDP在 给 定 策 略Pi下 的 状 态 价 值， 初 始 时 价 值 为V
    :param MDP:
    :param V:
    :param Pi:
    :param n:
    :return:
    """
    for i in range(n):
        V = update_V(MDP, V, Pi)
    return V

V = policy_evaluate(MDP, V, Pi, 10000)
display_dict(V)

浏览手机中: -2.31

第一节课: -1.31

第二节课: 2.69

第三节课: 7.38

休息中: 0.00



In [132]:
# 计算均一随机策略下，状态“第三节课”的最终价值
v = compute_v(MDP, V, Pi, "第三节课")
print("第 三 节 课 在 当 前 策 略 下 的 最 终 价 值 为:{:.2f}".format(v))

第 三 节 课 在 当 前 策 略 下 的 最 终 价 值 为:7.38


In [133]:
def compute_v_from_max_q(MDP, V, s):
    """
    根 据 一 个 状 态 的 下 所 有 可 能 的 行 为 价 值 中 最 大 一 个 来 确 定 当 前 状 态 价 值
    :param MDP:
    :param V:
    :param s:
    :return:
    """
    S, A, R, P , gamma = MDP
    v_s = -float('inf')
    for a in A:
        qsa = compute_q(MDP, V, s, a)
        if qsa >= v_s:
            v_s = qsa
    return v_s

In [134]:
def update_V_without_pi(MDP, V):
    """
    在 不 依 赖 策 略 的 情 况 下 直 接 通 过 后 续 状 态 的 价 值 来 更 新 状 态 价 值
    :param MDP:
    :param V:
    :return:
    """
    S, _, _, _, _ = MDP
    V_prime = V.copy()
    for s in S:
        # set_value(V_prime, s, compute_v_from_max_q(MDP, V_prime, s))
        V_prime[str_key(s)] = compute_v_from_max_q(MDP, V_prime, s)
    return V_prime

def value_iterate(MDP, V, n):
    """
    价值迭代
    :param MDP:
    :param V:
    :param n:
    :return:
    """
    for i in range(n):
        V = update_V_without_pi(MDP, V)
    return V

V= {}
# 通过价值迭代得到最优状态价值
V_star = value_iterate(MDP, V, 4)
display_dict(V_star)

浏览手机中: 6.00

第一节课: 6.00

第二节课: 8.00

第三节课: 10.00

休息中: 0.00



In [138]:
# 验 证 最 优 行 为 价 值
s, a = "第三节课", "泡吧"
q = compute_q(MDP, V_star, "第三节课", "泡吧")
print("在状态{}选择行为{}的最优价值为:{:.2f}".format(s,a,q))
# 将 输 出 结 果：
# 在 状 态 第 三 节 课 选 择 行 为 泡 吧 的 最 优 价 值 为:9.40

在状态第三节课选择行为泡吧的最优价值为:9.40
